In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_groq import ChatGroq
from base64 import b64decode
import os
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from base64 import b64decode
from tenacity import retry, stop_after_attempt, wait_exponential

In [ ]:
# Set your Groq API key
#os.environ["GROQ_API_KEY"] = "your-api-key-here"  # Replace with your actual key

# Document parser with error handling
def parse_docs(docs):
    """Extract text and base64 images from documents"""
    b64_images = []
    text_chunks = []

    for doc in docs:
        # Handle different document formats
        if isinstance(doc, Document):
            content = doc.page_content
        else:
            content = str(doc)

        # Validate base64 images
        try:
            if content.startswith("data:image/"):
                b64decode(content.split(",")[1])
                b64_images.append(content)
                continue
        except Exception:
            pass

        text_chunks.append(content)

    return {"images": b64_images, "texts": text_chunks}

# Prompt builder with validation
def build_prompt(inputs):
    """Construct LLM prompt from context and question"""
    if not isinstance(inputs, dict) or "context" not in inputs:
        raise ValueError("Invalid input format for prompt building")

    context_text = "\n".join(inputs["context"]["texts"])
    question = inputs["question"]



    ''' if len(docs_by_type["images"]) > 0:
        for image in docs_by_type["images"]:
            prompt_content.append(
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{image}"},
                }
            )'''




    return f"""You are an expert literary analyst. Answer the question using only the provided context.

Context:
{context_text}

Question: {question}

Answer in complete sentences, being careful to:
- Do not start with based on the provided context
- Cite specific details from the context
- do not assume anything about the context
- If unsure, state that information is not available
- Format your response clearly with proper paragraph breaks"""

# Robust chain construction
chain = (
    {
        "context": RunnableLambda(parse_docs),  # Directly process retriever output
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(build_prompt)
    | ChatGroq(
        model_name="llama3-70b-8192",  # More capable than 8B model
        temperature=0.2,               # More factual responses
        max_tokens=1024,
        max_retries=3,                # Built-in retry mechanism
    )
    | StrOutputParser()
)

# Retry decorator for API calls
@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=2, max=30),
    reraise=True
)
def get_answer(question):
    """Safe execution with error handling"""
    try:
        return chain.invoke(question)
    except Exception as e:
        print(f"Error processing question: {str(e)}")
        raise



In [ ]:
question="The Ministry of Industry and Information Technology (MIIT) is responsible for?"
response = get_answer(question)
print("\nFinal Answer:")
print(response)

In [ ]:
question="Chinas ICT system consists of?"
response = get_answer(question)
print("\nFinal Answer:")
print(response)